The following cell sets the input values required to run the the optimisation model ``..\MINIZINC\hydrogen_plant.mzn``.

This input data are stored in ``..\MINIZINC\hydrogen_plant_data.dzn``. This input file preparation is carried out by ``make_dzn_file`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``.

Then the minizinc optimisation is executed by ``optimise()`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``. 

``projdirs`` stores the key paths for the entire package.

### Set the location and get the solar and wind source data

In [ ]:
import pandas as pd
import numpy as np
import os
from projdirs import datadir #load the path that contains the data files 
from PACKAGE.optimisation import make_dzn_file, optimise
from PACKAGE.component_model import WindSource, SolarResource,pv_gen, wind_gen,solcast_weather, SolarResource_solcast, WindSource_solcast

# Set the location

#Newman
Newman = [-23.35, 119.75]
Sydney = [-33.856784, 151.215297]
Tom_Price = [-22.69, 117.79]
Port_Augusta = [-32.49, 137.77]
Whyalla = [-33.04, 137.59]
Pinjarra = [-32.63, 115.87]
Gladstone = [-23.84, 151.25]
Burnie = [-41.05, 145.91]

# Get wind and solar data for the designated location
# WindSource(Lat, Lon)
# SolarResource(Lat,Lon)


# solcast_weather(Newman)
SolarResource_solcast()
WindSource_solcast()

### Set the inputs for plant optimisation and run the optimisation

In [ ]:
pv_ref_capa = 1e6 #(W)
pv_ref_out = pv_gen(pv_ref_capa)

wind_ref_capa = 50e6 #(W)
wind_ref_out = wind_gen()

def AUD2USD(value):
    return(0.746 * value)

# create a dictionary that contains the inputs for optimisation.
#these inputs are used by make_dzn_file function to create an input text file called hydrogen_plant_data.dzn. 
simparams = dict(DT = 3600,#[s] time steps
                 ETA_EL = 0.70,       #efficiency of electrolyser
                 BAT_ETA_in = 0.95,   #charging efficiency of battery
                 BAT_ETA_out = 0.95,  #discharg efficiency of battery
                 C_PV = AUD2USD(1.258),          #[$/W] unit cost of PV
                 C_W = AUD2USD(1.934),           #[$/W] unit cost of W
                 C_EL = 0.835,          #[$/W] unit cost of electrolyser
                 C_HS = 95,           #[$/kgH] unit cost of hydrogen storage
                 C_BAT_energy = AUD2USD(400/1000/3600),        #[$/W.s] unit cost of battery energy storage
                 C_BAT_power = 60/1000,        #[$/W] unit cost of battery power capacpity
                 CF = 0.95,           #capacity factor
                 PV_ref_capa = pv_ref_capa,    #capacity of reference PV plant (W)
                 PV_ref_out = pv_ref_out,           #power output from reference PV plant (W)
                 Wind_ref_capa = wind_ref_capa,      #capacity of reference wind farm (W)
                 Wind_ref_out = wind_ref_out,        #power output from the reference wind farm (W)
                 L = [0.1 for i in range(len(pv_ref_out))])       #[kgH2/s] load profile timeseries

#run the optimisation function and get the results in a dictionary:
results = optimise(simparams)

#refine the cost of storage
from numpy import log10
def c_hs(hs_max):
    if hs_max>20:
        cost=10 ** (0.212669*(log10(hs_max))**2 - 1.638654*(log10(hs_max)) + 4.403100)
    else:
        cost = 10 ** (-0.0285*log10(hs_max)  + 2.7853)
    return(cost)

while abs(simparams['C_HS'] - c_hs(results['hs_max'][0]/1e3))/simparams['C_HS'] > 0.05:
    print('Hydrogen storage capacity = %0.2f [T of H2]' %( results['hs_max'][0]/1e3 ))
    new_C_HS = c_hs(results['hs_max'][0]/1e3)
    print('new C_HS=', new_C_HS)
    simparams['C_HS'] = new_C_HS
    results = optimise(simparams)
          
          
print (
        'CAPEX = %0.2f [M$]' %( results['CAPEX'][0]/1e6 ),'\n'
        'PV rated capacity  = %0.2f [MW]' %( results['pv_max'][0]/1e6 ),'\n'  
        'Wind rated capacity = %0.2f [MW]' %( results['w_max'][0]/1e6 ),'\n'    
        'Electrolyser rated capacity = %0.4f [MW]' %( results['el_max'][0]/1e6 ),'\n'  
        'Hydrogen storage capacity = %0.2f [T of H2]' %( results['hs_max'][0]/1e3 ),'\n'  
        'Battery energy capacity = %0.2f [MWh]' %( results['bat_capa'][0]/3.6e9  ),'\n'  
        'Battery power capcity = %0.2f [MW]' %( results['bat_pmax'][0]/1e6 )
        )


### Post process data

In [ ]:
path = r'C:\Nextcloud\HILT-CRC---Green-Hydrogen\DATA\SAM INPUTS\WEATHER_DATA'
weather_data = pd.read_csv(path + "\weather_data.csv",skiprows=2)


beam = weather_data['DNI'].tolist()
G_H = weather_data['GHI']
wind_speed = weather_data['Wind Speed']

water_rate = 15 #L/kgH2  range: 15-20 L/kgH2
c_wate = 5 # $/m3

#transfer the data from 'results' to a dataframe for plotting: 
data_plot = pd.DataFrame(dict(t = np.arange(0,len(simparams['L'])),
                              beam = beam,
                              G_H = G_H,
                              wind_speed = wind_speed,
                              pv_out=results['pv_out']/1e6, #MW
                              w_out=results['w_out']/1e6, #MW
                              pc=results['pc']/1e6, #MW
                              he_out = results['he_out']*3.6, #T H2/hr
                              hr_out = results['hr_out']*3.6, #T H2/hr
                              sh = results['sh'][0:-1]/1e3, #T H2
                              R = results['r'][0:-1]/1e3, #T H2/hr 
                              L = results['L']*3.6, #T H2/hr
                              hs_out = (results['L']-results['hr_out'])*3.6, #T H2/hr
                              bat_e_stored = results['bat_e_stored'][0:-1]*3.6, #MWh
                              bat_pin = results['bat_pin']/1e6, #MW
                              bat_pout = results['bat_pout']/1e6, #MW
                              el_pin = results['el_pin']/1e6, #MW
                              water_mdot = results['he_out']*3.6*water_rate,  #T of H2O/hr
                              ))

#create a new column for datetime for plotting. This column is timestamped from the 
#beginning of 2022
data_plot['time'] =pd.to_datetime('2021-01-01') + pd.to_timedelta(data_plot.t, 'h')
data_plot.drop('t',axis=1,inplace=True)


## Plot solar irradiance and wind speed

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.beam),
              line_shape='hv', name='Direct Solar Irrad. [W/m<sup>2</sup>]',
               line_color = 'red', opacity= 0.5, yaxis="y1"),
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.G_H),
              line_shape='hv', name='Global H. Solar Irrad. [W/m<sup>2</sup>]',
               line_color = 'orange', opacity= 0.5, yaxis="y1"),
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.wind_speed),
              line_shape='hv', name='Wind Speed [m/s]',
               line_color = 'green', opacity= 0.5, yaxis="y2"),
                )

fig.update_yaxes(dict(title = 'W/m<sup>2</sup>'),
                 linecolor='black',
                 mirror=True, secondary_y=False)

fig.update_yaxes(dict(title = 'm/s'), secondary_y=True)

fig.update_xaxes(linecolor='black',mirror=True)


fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=20, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\PVW-CF=%s.png'%str(simparams['CF']))

fig.show()

### Plot mass flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.sh),
              name='H<sub>2</sub> Stored [T]',yaxis="y1"),
                )
# fig.add_trace(
#     go.Scatter(x=list(data_plot.time), y=list(data_plot.R),
#               name='R',yaxis="y1")
#                 )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.he_out),
              line_shape='hv',name='H<sub>2</sub> from Electrolyser [T/hr]',
               yaxis="y2", opacity=0.5)
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hs_out),
              line_shape='hv',name='H<sub>2</sub> from Storage [T/hr]',yaxis="y2")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hr_out),
              line_shape='hv',name='Unserved Load [T/hr]',yaxis="y2")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.water_mdot),
              line_shape='hv',name='Water to Electrolyser [T/hr]',
               opacity = 0.5, yaxis="y2")
                )

fig.update_yaxes(dict(title = '<b>T of H<sub>2</sub></b>',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = '<b>Tonne/hr</b>',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=30, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Hflow-CF=%s.png'%str(simparams['CF']))
fig.show()

### Plot power flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pv_out),
              line_shape='hv', name='PV Generation [MW]',
              line_color = 'orange', opacity= 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.w_out),
              line_shape='hv', name='Wind Generation [MW]',
              line_color = 'green', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pc),
              line_shape='hv', name='Curtailed Power [MW]',
              line_color = 'grey', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pin),
              line_shape='hv', name='Battery Charging [MW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pout),
              line_shape='hv', name='Battery Discharging [MW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.el_pin),
              line_shape='hv', name='Electrolyer Pin [MW]', yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_e_stored),
              line_shape='hv', name='Battery Energy [MWh]', yaxis="y2"),
                )


fig.update_yaxes(dict(title = 'MW',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = 'MWh',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=0, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Pflow-CF=%s.png'%str(simparams['CF']))

fig.show()